# Project 1 - EEG

In [14]:
import dlc_bci as bci

import torch
from torch.autograd import Variable
from torch.nn import functional as F
from torch import Tensor

In [2]:
train_input , train_target = bci.load(root = './data_bci', one_khz=True)

print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])


In [3]:
test_input, test_target = bci.load(root = './data_bci', train=False, one_khz=True)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


## Defining the model

In [4]:
n_channels = 32
kernel_size = 3
n_hidden = 128
input_shape = train_input.shape
ff1_in = n_channels * (input_shape[2] - 2) // 2


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv1d(input_shape[1], n_channels, kernel_size)
        self.max_pool = torch.nn.MaxPool1d(2)
        self.fc1 = torch.nn.Linear(ff1_in, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, 2)
        self.sm = torch.nn.Softmax(1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = x.view(x.shape[0], ff1_in)
        x = F.relu(self.fc1(x))
        x = self.sm(self.fc2(x))
        return x

In [5]:
def train_model(model, train_input, train_target, mini_batch_size, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    eta = 1e-1

    for e in range(epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.data[0]
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)

In [21]:
def train_model(model, train_input, train_target, mini_batch_size, n_epochs, verbose=0):
    criterion = torch.nn.CrossEntropyLoss()
    eta = 1e-1
    
    for e in range(n_epochs):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.data[0]
            model.zero_grad()
            loss.backward()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        if verbose:
            print(e, sum_loss)

In [19]:
def compute_nb_errors(model, input, target, mini_batch_size):

    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = output.data.max(1)[1]
        true_classes = target.data.narrow(0, b, mini_batch_size)
        nb_errors += (predicted_classes == true_classes).sum()

    return nb_errors

In [7]:
train_input, train_target = Variable(train_input), Variable(train_target.long())
test_input, test_target = Variable(test_input), Variable(test_target.long())

In [23]:
mini_batch_size = 4
epochs = 25

for k in range(0, 10):
    model = Net()
    train_model(model, train_input, train_target, mini_batch_size, epochs, verbose=True) 
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('Test error: {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                       nb_test_errors, test_input.size(0)))

0 64.30404776334763
1 64.49767112731934
2 64.49767112731934
3 64.49767112731934
4 64.49767112731934
5 64.49767112731934
6 64.49767112731934
7 64.49767112731934
8 64.49767112731934
9 64.49767112731934
10 64.49767112731934
11 64.49767112731934
12 64.49767112731934
13 64.49767112731934
14 64.49767112731934
15 64.49767112731934
16 64.49767112731934
17 64.49767112731934
18 64.49767112731934
19 64.49767112731934
20 64.49767112731934
21 64.49767112731934
22 64.49767112731934
23 64.49767112731934
24 64.49767112731934
test error Net 51.00%% 51/100
0 64.51877546310425
1 64.49767112731934
2 64.49767112731934
3 64.49767112731934


KeyboardInterrupt: 

In [24]:
out = model.forward(train_input[[4,5,6,7,8,21,32,53]])
out

Variable containing:
    0     1
    0     1
    0     1
    0     1
    0     1
    0     1
    0     1
    0     1
[torch.FloatTensor of size 8x2]